In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 17.0MB/s 
     |████████████████████████████████| 1.1MB 40.0MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
     |████████████████████████████████| 133kB 56.0MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 7.1MB 60.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ed5eb2f365e0f520dcf5db0f7798c95c79d82e1f71a1e2caf1a970eb68836f3a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.37 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd

test = pd.read_csv('/content/gdrive/MyDrive/test_우_d_중_긍정.csv', encoding = 'utf-8')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(text, test_size=0.2,
                                                    shuffle=True, random_state=1004)

In [ ]:
!git clone https://github.com/e9t/nsmc.git

train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')


Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.69 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
train = train.reset_index()

In [ ]:
train.rename(columns = {'idex': 'id','content':'text', 'tag':'label'}, inplace = True)
train

,index,text,label
0,892,이미 또 터졌 택시 기사 어쩌 아이 학교 불러들이 부모 나라 국민 도저히 도저히 이...,0
1,388,코로나 19 항체치료제가 생산 계획에 있다는 소식만으로도 정말 다행이라고 생각합니다...,1
2,3958,서구의 글로벌 회사들의 치료제. 백신은 국적과 무관하게 모두 미 FDA 승인을 받는...,2
3,5041,ㅎㄷㄷ,2
4,5429,쟤내는 멀해도 안되ㅋㅋ,2
...,...,...,...
4715,199,파이팅,1
4716,1263,서울 출장 가야 정말 가 싫,0
4717,5445,양심은있네 빨리 백신좀 만들자 이제,2
4718,4127,제발 요금 정산해라... 비행기 타고 온 거부터 모조리 싹 다,2


In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [ ]:
nsmc_train_dataset = NsmcDataset(train)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
device = torch.device('cuda')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.to(device)

100%|██████████| 714314041/714314041 [00:08<00:00, 81513866.67B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 2
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/2] Iteration 500 -> Train Loss: 0.9055, Accuracy: 0.502
[Epoch 1/2] Iteration 1000 -> Train Loss: 0.6913, Accuracy: 0.748
[Epoch 1/2] Iteration 1500 -> Train Loss: 0.4867, Accuracy: 0.868
[Epoch 1/2] Iteration 2000 -> Train Loss: 0.3913, Accuracy: 0.887
[Epoch 2/2] Iteration 2500 -> Train Loss: 0.3109, Accuracy: 0.909
[Epoch 2/2] Iteration 3000 -> Train Loss: 0.3464, Accuracy: 0.885
[Epoch 2/2] Iteration 3500 -> Train Loss: 0.2926, Accuracy: 0.901
[Epoch 2/2] Iteration 4000 -> Train Loss: 0.2917, Accuracy: 0.898
[Epoch 2/2] Iteration 4500 -> Train Loss: 0.2461, Accuracy: 0.911


In [ ]:
del test['Unnamed: 0']
test

,index,text,label
0,5839,무조건 중국 막 시작 문재인 라고 씨부리 과학 대응 이미 죽 애도 표한다 대통령 죽 다,0
1,5388,일본 대한 호랑이 물똥 산 나라 아이 비 많이 오 씻겨 사라질 이 역사 왜곡 후손 ...,0
2,12758,말 하 말 못 해,0
3,5132,시국 제주 여행 가 좋 민폐 족자 코로나 폐렴 병 걸려 죽,0
4,2819,하 또래 제발 클럽 좀 가지 시국 가 비비 댕기 바로 확진 인디 생각 좀 하 제발,0
...,...,...,...
2221,5774,개합 아 정신 안 차릴 레,0
2222,5105,등교 전 학교 방문 방역 문재인 유은혜 사진 존나 찍 지금 내성 이 학교 가 안 전...,0
2223,9515,아 진짜 보 드문 잡것 중국 우한 디렉트 우한 폐렴 수입 국민 통제 시키 시봉 타이...,0
2224,8989,북한 사살 후 시신 불태우 방법 훼손 배경 대해 서 장관 지금 북한 상황 정확히 말...,0


In [ ]:
test = test.reset_index()

test.rename(columns = {'idex': 'id','content':'text', 'tag':'label'}, inplace = True)
test

,index,text,label
0,4683,군법 파괴자~~정답!!!,2
1,2979,"개신교 다 비대면 예배 하겠다고 발표했고 ,1.2월부터 온라인 예배드리고 있어요.전...",2
2,678,맞벌이 라 얼 집 긴급 보육 다니 막내 넘 걱정 가게 접 없,0
3,407,장기전 될 같 입국 금지 해외 유입 막 내수 라도 살려야 입국 제한 제발 한국 일상...,0
4,4086,검사 수가 적어 저 정도지 엄청날 것입니다시 한 폭탄,2
...,...,...,...
1176,1538,이럴 때 아니 또 퍼진다,0
1177,404,아 개학 얼마 안 남 어쩌 긴 연기 점점 조심 곳곳 무뎌져 끝 긴장 놓치 안 되,0
1178,1878,제발 좀 자국민 좀 살려도,0
1179,2017,중국 세계 사죄 피해 보상 소중 일상 다 망가졌,0


In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.0


In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/긍우중_epoch_2_itr.pth')

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/긍우중_all_epoch_2_itr.pth')

In [ ]:
train.to_csv('/content/gdrive/MyDrive/train_우_d_중.csv', encoding = 'utf-8-sig')

In [ ]:
test.to_csv('/content/gdrive/MyDrive/test_우_d_중.csv', encoding = 'utf-8-sig')

In [ ]:
model = torch.load('/content/gdrive/MyDrive/긍우중_all_epoch_4_itr.pth')

## 저장된 모델 불러와 추가 학습시키기

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

100%|██████████| 995526/995526 [00:00<00:00, 33040860.01B/s]


In [ ]:
device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 2
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/2] Iteration 500 -> Train Loss: 0.2505, Accuracy: 0.917
[Epoch 1/2] Iteration 1000 -> Train Loss: 0.2479, Accuracy: 0.916
[Epoch 1/2] Iteration 1500 -> Train Loss: 0.2273, Accuracy: 0.925
[Epoch 1/2] Iteration 2000 -> Train Loss: 0.1952, Accuracy: 0.938
[Epoch 1/2] Iteration 2500 -> Train Loss: 0.1734, Accuracy: 0.943
[Epoch 2/2] Iteration 3000 -> Train Loss: 0.1893, Accuracy: 0.942
[Epoch 2/2] Iteration 3500 -> Train Loss: 0.1432, Accuracy: 0.955
[Epoch 2/2] Iteration 4000 -> Train Loss: 0.1976, Accuracy: 0.938
[Epoch 2/2] Iteration 4500 -> Train Loss: 0.2004, Accuracy: 0.938
[Epoch 2/2] Iteration 5000 -> Train Loss: 0.1678, Accuracy: 0.951
[Epoch 2/2] Iteration 5500 -> Train Loss: 0.1824, Accuracy: 0.941


In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/긍우중_epoch_4_itr.pth')

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/긍우중_all_epoch_4_itr.pth')